In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [ ]:
! pip install -qU langchain-openai

In [ ]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = ""
os.environ['LANGCHAIN_ENDPOINT'] = ""
os.environ['LANGCHAIN_API_KEY'] = ""

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from pathlib import Path
from pprint import pprint

# SQL AGENT

In [ ]:
import pandas as pd

df = pd.read_csv("High Risk Profiles.csv")
df1= pd.read_csv("Low Risk Profiles.csv")

print(df.shape)
print(df.columns.tolist())
print(df1.shape)
print(df1.columns.tolist())

In [ ]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///risk.db")
df.to_sql("high_risk", engine, index=False)
df1.to_sql("low_risk", engine, index=False)

In [ ]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM high_risk WHERE claim = 0;"))

In [ ]:
llm = ChatOpenAI(model="gpt-4")

In [ ]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
response = agent_executor.invoke({"input": "list two rows where claim = 1 and risk is high"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
response = output_parser.parse(response)

# Profile Report tool

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
prompt_template = """
Given the following details of a potential customer's auto insurance profile:\n\n
Context:\n {context}?\n
Analyze and provide a cocise summary of the insights that can be derived from this data regarding the customer's risk profile and potential impact on insurance strategy. Consider aspects such as premium pricing, risk mitigation, and strategic considerations for insurance policy adjustments based on the provided data.
Make sure you keep it concise and clear.
Answer:
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_template)

model = ChatOpenAI(model_name="gpt-4", temperature=0.3)

In [ ]:
context = {'claim_likelihood': 0.07961550057185947,
 'claim_amount': 6434.927335660211,
 'normalized_claim_likelihood': 0.07556854055743428,
 'normalized_claim_amount': 0.19096796236790398,
 'risk_profile_probability': 'Medium',
 'risk_profile_cost': 'Low',
 'dynamic_combined_risk_score': 0.11417145522587874,
 'risk_group': 'High Risk'}

In [ ]:
chain = prompt | model | StrOutputParser()

In [ ]:
response = chain.invoke({"context": context})

In [ ]:
print(response)

# Reports RAG tool

In [ ]:
%pip install pypdf

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "Demographic Analysis.pdf"

loader = PyPDFLoader(file_path)
pages = loader.load()

len(pages)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
splits = text_splitter.split_documents(pages)

In [ ]:
splits[0]

In [ ]:
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings())

In [ ]:
question = "Give me the relation between age and risk group"
answer = vectorstore.similarity_search(question)

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
prompt_template = """
Answer the question as detailed as possible from the provided context and make conclusions in a non technical manner, make sure to provide all the details, if the answer is not in
provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
Context:\n {context}?\n
Question: \n{question}\n

Answer:
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI(model_name="gpt-4", temperature=0.3)

In [ ]:
chain = (
# The initial dictionary uses the retriever and user supplied query
   prompt  | model | StrOutputParser()

)

In [ ]:
chain.invoke(question)